In [ ]:
!pip install transformers datasets torch scikit-learn


In [2]:
import torch
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import Dataset
import random


In [3]:
def create_sample_data(num_samples=1000):
    positive_reviews = [
        "Professor Smith is an excellent teacher who explains concepts clearly.",
        "Amazing instructor, very helpful during office hours.",
        "The best teacher I've had in college, highly recommend.",
        "Clear explanations and fair grading system.",
        "Engaging lectures that make the material interesting.",
    ]

    negative_reviews = [
        "Terrible teacher, doesn't explain anything clearly.",
        "Avoid this professor if possible, very unhelpful.",
        "The worst instructor I've ever had.",
        "Unclear expectations and unfair grading.",
        "Boring lectures that put everyone to sleep.",
    ]

    reviews, labels = [], []
    for _ in range(num_samples // 2):
        reviews.append(random.choice(positive_reviews))
        labels.append(1)
    for _ in range(num_samples // 2):
        reviews.append(random.choice(negative_reviews))
        labels.append(0)

    return pd.DataFrame({'text': reviews, 'label': labels})

df = create_sample_data(800)


In [ ]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df['text'].tolist(),
    df['label'].tolist(),
    test_size=0.3,
    random_state=42,
    stratify=df['label']
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts,
    temp_labels,
    test_size=0.5,
    random_state=42,
    stratify=temp_labels
)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels}).map(tokenize_function, batched=True)
val_dataset   = Dataset.from_dict({'text': val_texts,   'label': val_labels}).map(tokenize_function, batched=True)
test_dataset  = Dataset.from_dict({'text': test_texts,  'label': test_labels}).map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/560 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2
)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir='./logs',
    logging_steps=10,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

In [ ]:
model.save_pretrained("./teacher_review_model")
tokenizer.save_pretrained("./teacher_review_model")


('./teacher_review_model/tokenizer_config.json',
 './teacher_review_model/special_tokens_map.json',
 './teacher_review_model/vocab.txt',
 './teacher_review_model/added_tokens.json')

In [ ]:
def predict_sentiment(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probs, dim=1).item()
    confidence = probs[0][predicted_class].item()

    sentiment = "Positive" if predicted_class == 1 else "Negative"
    return {"sentiment": sentiment, "confidence": confidence}


In [ ]:
print(predict_sentiment(" what the fuck the teacher taught i couldn't understand it "))


{'sentiment': 'Negative', 'confidence': 0.9982584118843079}
